In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma import aims
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
a = ana.Anatomist()

global modules:create qapp
 /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module meshsplit
loading module palettecontrols
loading module volumepalettes


existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


### Variable definitions

In [3]:
embeddings_file = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0_trained_on_UKB40/UKB-RAP/right_handed_individual_prediction_all_regions.csv"
qc_file = "/neurospin/dico/data/deep_folding/current/datasets/UkBioBank40/qc.tsv"

In [4]:
label = "qc"

In [5]:
side = "R" # "R" or "L"
region = "S.C.-S.Pe.C."# "S.C.-sylv.", "ORBITAL", "CINGULATE", "SC-sylv", "F.I.P."
region_s = "SC-SPeC_right"
database='UkBioBank'

# Building predictors

In [6]:
participants = pd.read_csv(qc_file, dtype={'participant_id':object}, sep='\t')
participants = participants.set_index("participant_id")
participants = participants[participants.qc==1][["qc"]]
participants.head()

,qc
participant_id,
sub-1000021,1
sub-1000325,1
sub-1000458,1
sub-1000575,1
sub-1000606,1


In [7]:
participants.head()

,qc
participant_id,
sub-1000021,1
sub-1000325,1
sub-1000458,1
sub-1000575,1
sub-1000606,1


In [8]:
# participants = participants[[label]].dropna()
# participants = participants.replace({"N": 0, "?": 1, "Y": 2})
participants["qc"].unique()

array([1])

In [9]:
len(participants)

42433

In [10]:
embeddings = pd.read_csv(embeddings_file, dtype={'ID':object})
embeddings = embeddings.set_index("ID")
embeddings = embeddings.loc[:, [region_s]].rename(columns={region_s: "predicted"})
embeddings.head()

,predicted
ID,
sub-1000606,0.942130
sub-1000715,0.923639
sub-1000963,0.918711
sub-1001107,0.882049
sub-1002539,0.901626


In [11]:
merged = participants[[label]].merge(embeddings, left_index=True, right_index=True)

df = merged.copy()

In [12]:
df.mean()

qc           1.000000
predicted    0.905322
dtype: float64

In [13]:
df.head()

,qc,predicted
sub-1000021,1,0.918917
sub-1000325,1,0.883078
sub-1000458,1,0.919036
sub-1000575,1,0.893553
sub-1000606,1,0.942130


### Function definitions

In [14]:
def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

def buckets_average(subject_id_list, dataset_name_list):
    dic_vol = {}
    dim = 0
    rep = 0
    if len(subject_id_list) == 0:
        return False
    while dim == 0 and rep < len(subject_id_list):
        if dataset_name_list[rep].lower() in ['ukb', 'ukbiobank', 'projected_ukb']:
            dataset = 'UkBioBank40'
        elif dataset_name_list[rep].lower() in ['hcp']:
            dataset = 'hcp'
        elif dataset_name_list[rep].lower() in ['imagen']:
            dataset = 'imagen'
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"
        if os.path.isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
            sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
            dim = sum_vol.shape
            sum_vol.fill(0)
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
        rep += 1

    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() in ['ukb', 'ukbiobank',  'projected_ukb']:
            dataset = 'UkBioBank40'
        elif dataset.lower() == 'hcp':
            dataset = 'hcp'
            
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

        if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
            vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            # compare the dim with the first file dim

            if vol.np.shape != dim:
                raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

                
            # to have a binary 3D structure
            dic_vol[subject_id] = (vol.np > 0).astype(int)
            sum_vol.np[:] += (vol.np > 0).astype(int) 
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

    sum_vol = sum_vol / len(subject_id_list)
    print(sum_vol, sum_vol.shape)
    return sum_vol

In [15]:
def visualize_averages_along_parameter(df, column_name, database, nb_subjects_per_average=1500, nb_columns=3, nb_lines=1):
    # anatomist objects
    global _block
    global _average_dic
    global dic_packages
    nb_windows = nb_columns * nb_lines
    _average_dic = {}
    step = nb_subjects_per_average # number of subjects on which average is done
    # Creates the block if it has not been created
    try:
        _block
    except NameError:
        _block = a.createWindowsBlock(nb_columns)

    # Order according to column_name
    df = df.sort_values(column_name)
    predict_proba = df[[column_name]]

    # Creates dictionary of subjects to average
    dic_packages = {}
    for i in range(0,len(predict_proba),step):
        list_idx = (predict_proba.index[i:i+step].to_numpy())
        dic_packages[i//step] = list_idx
        
    # Ensures that last list contains the last step subjects
    list_idx = (predict_proba.index[-step:].to_numpy())
    dic_packages[i//step] = list_idx
        
    list_database = [database for i in range(step)]
    n_pack = len(dic_packages)

    # Loop of display averages
    list_pack = [int(np.ceil(i*n_pack/float(nb_windows))) for i in range(0,nb_windows)]
    list_pack[-1] = n_pack-1
    for i in list_pack:
        sum_vol = buckets_average(dic_packages[i], list_database)
        _average_dic[f'a_sum_vol{i}'] = a.toAObject(sum_vol)
        _average_dic[f'a_sum_vol{i}'].setPalette(minVal=0, absoluteMode=True)
        #wsum = a.createWindow('Sagittal', block=block)
        #wsum.addObjects(a_sum_vol)
        _average_dic[f'rvol{i}'] = a.fusionObjects(
            objects=[_average_dic[f'a_sum_vol{i}']],
            method='VolumeRenderingFusionMethod')
        _average_dic[f'rvol{i}'].releaseAppRef()
        # custom palette
        n = len(dic_packages[i])
        pal = a.createPalette('VR-palette')
        pal.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
        build_gradient(pal)
        _average_dic[f'rvol{i}'].setPalette('VR-palette', minVal=0.05, absoluteMode=True)
        pal2 = a.createPalette('slice-palette')
        pal2.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
        build_gradient(pal2)
        _average_dic[f'a_sum_vol{i}'].setPalette('slice-palette')
        # rvol.palette().fill()
        _average_dic[f'wvr{i}'] = a.createWindow('3D', block=_block)
        _average_dic[f'wvr{i}'].addObjects(_average_dic[f'rvol{i}'])

### Visualization

In [16]:
# block = a.createWindowsBlock(10)
visualize_averages_along_parameter(df, "predicted", database)

<soma.aims.Volume_DOUBLE object at 0x756ee6117f40> (42, 47, 51, 1)
Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram fo